In [1]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

# Statistics of US Businesses  
Census Bureau Product, data download homepage is here: https://www.census.gov/programs-surveys/susb/data/tables.html  
Currently using county by state 2020: https://www.census.gov/data/tables/2020/econ/susb/2020-susb-annual.html  
This is for 2020 released in 2023.  
When downloaded, open the Excel workbook and delete the top two rows (the title and the notes) so that we are left with the row of column headers at the top. Copy-paste the content into an unformatted csv, and save in Data Downloads as "StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2020.csv", etc. for different years. When you go back to 2015 there is both establishment and enterprise industry, select establishment industry.

In [30]:
twentytwenty = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2020.csv', dtype = 'str')
twentytwenty = twentytwenty.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentytwenty = twentytwenty.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentytwenty['Year'] = '2020'
twentytwenty.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
0,01,Alabama,001,Autauga,--,Total,1: Total,807,879,11265,385785,2020
1,01,Alabama,001,Autauga,--,Total,2: <20 employees,520,522,2234,69724,2020


In [31]:
twentynineteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2019.csv', dtype = 'str')
twentynineteen = twentynineteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentynineteen = twentynineteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentynineteen['Year'] = '2019'
twentynineteen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
0,01,Alabama,001,Autauga,--,Total,1: Total,797,867,11510,385755,2019
1,01,Alabama,001,Autauga,--,Total,2: <20 employees,517,520,2272,68970,2019


In [32]:
twentyeighteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2018.csv', dtype = 'str')
twentyeighteen = twentyeighteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag'])
twentyeighteen = twentyeighteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentyeighteen['Year'] = '2018'
twentyeighteen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
0,01,Alabama,001,Autauga,--,Total,1: Total,785,855,11397,373865,2018
1,01,Alabama,001,Autauga,--,Total,2: <20 employees,509,510,2245,67131,2018


In [36]:
#this year they started including "Employment Range Flag" so drop that column
twentyseventeen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2017.csv', dtype = 'str')
twentyseventeen = twentyseventeen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag', 
                                                  'Receipts\n($1,000)', 'Receipts\nNoise Flag'])
twentyseventeen = twentyseventeen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentyseventeen['Year'] = '2017'
twentyseventeen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
0,01,Alabama,001,Autauga,--,Total,1: Total,800,869,11036,360936,2017
1,01,Alabama,001,Autauga,--,Total,2: <20 employees,530,531,2379,70085,2017


In [38]:
twentysixteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2016.csv', dtype = 'str')
twentysixteen = twentysixteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
twentysixteen = twentysixteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentysixteen['Year'] = '2016'
twentysixteen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
0,01,ALABAMA,001,AUTAUGA,--,Total,1: Total,791,851,10790,332497,2016
1,01,ALABAMA,001,AUTAUGA,--,Total,2: <20 employees,519,520,2303,64084,2016


In [42]:
twentyfifteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2015.csv', dtype = 'str')
twentyfifteen = twentyfifteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employmment Range Flag'])
twentyfifteen = twentyfifteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
twentyfifteen['Year'] = '2015'
twentyfifteen.head(2)

,State,State Name,County,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year
0,01,ALABAMA,001,AUTAUGA,--,Total,1: Total,788,844,10454,321433,2015
1,01,ALABAMA,001,AUTAUGA,--,Total,2: <20 employees,526,527,2216,63553,2015


## They started doing NAICS breakouts at the county level in 2015, so from here back it's only total

In [55]:
# twentyfourteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2014.csv', dtype = 'str')
# twentyfourteen = twentyfourteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentyfourteen = twentyfourteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentyfourteen['Year'] = '2014'
# twentyfourteen.head(2)

In [56]:
# twentythirteen = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2013.csv', dtype = 'str')
# twentythirteen = twentythirteen.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentythirteen = twentythirteen.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentythirteen['Year'] = '2013'
# twentythirteen.head(2)

In [57]:
# twentytwelve = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2012.csv', dtype = 'str')
# twentytwelve = twentytwelve.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentytwelve = twentytwelve.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentytwelve['Year'] = '2012'
# twentytwelve.head(2)

In [58]:
# twentyeleven = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2011.csv', dtype = 'str')
# twentyeleven = twentyeleven.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentyeleven = twentyeleven.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentyeleven['Year'] = '2011'
# twentyeleven.head(2)

In [59]:
# twentyten = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2010.csv', dtype = 'str')
# twentyten = twentyten.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twentyten = twentyten.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twentyten['Year'] = '2010'
# twentyten.head(2)

In [60]:
# twothounine = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2009.csv', dtype = 'str')
# twothounine = twothounine.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twothounine = twothounine.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothounine['Year'] = '2009'
# twothounine.head(2)

In [61]:
# twothoueight = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2008.csv', dtype = 'str')
# twothoueight = twothoueight.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# twothoueight = twothoueight.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothoueight['Year'] = '2008'
# twothoueight.head(2)

In [62]:
# twothouseven = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2007.csv', dtype = 'str')
# twothouseven = twothouseven.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag', 
#                                             'Receipts', 'Receipts Noise Flag'])
# twothouseven = twothouseven.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothouseven['Year'] = '2007'
# twothouseven.head(2)

In [63]:
# twothousix = pd.read_csv('../../Data Downloads/StatisticsofUSBusinesses_CountyAnnual3DigNAICS_2006.csv', dtype = 'str')
# #twothousix = twothousix.drop(columns = ['Employment Noise Flag', 'Annual Payroll Noise Flag', 'Employment Range Flag'])
# #twothousix = twothousix.rename(columns = {'Annual Payroll\n($1,000)': 'Annual Payroll ($1000)'})
# twothousix['Year'] = '2006'
# twothousix.head(2)

## Merging all dataframes with the NAICS breakout, read in whatever you need later.

In [64]:
dfs = [twentytwenty, twentynineteen, twentyeighteen, twentyseventeen, twentysixteen, twentyfifteen]

In [108]:
data = pd.concat(dfs).reset_index(drop = True)

In [109]:
data = data.rename(columns = {'State': 'StateFIPS', 'County': 'CountyFIPS', 'County Name': 'County'})

In [110]:
data['GEO_ID'] = '0500000US' + data['StateFIPS'] + data['CountyFIPS']

In [111]:
data = data.loc[data['StateFIPS'] == '47'].reset_index(drop = True)

In [112]:
data = data.drop(columns = ['StateFIPS', 'State Name', 'CountyFIPS'])

In [115]:
naicsdict = dict(zip(data['NAICS'], data['NAICS Description']))
countydict = dict(zip(data['County'], data['GEO_ID   ']))

In [113]:
data.head()

,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
0,Anderson,--,Total,1: Total,1305,1532,42762,2831834,2020,0500000US47001
1,Anderson,--,Total,2: <20 employees,861,870,3797,146123,2020,0500000US47001
2,Anderson,--,Total,3: 20-99 employees,145,165,5113,250992,2020,0500000US47001
3,Anderson,--,Total,4: 100-499 employees,70,112,3613,202320,2020,0500000US47001
4,Anderson,--,Total,5: 500+ employees,229,385,30239,2232399,2020,0500000US47001


In [ ]:
data = data.set_index(['County Name', ''])

In [76]:
#export to the SQLite database
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('SUSB_CountyAnnual_2015to2020', conn, if_exists = 'replace', index = False)

3005018